In [125]:
import requests
import urllib.request
import lxml.html as lh
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np
import geopy
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
print("Libraries imported")

Libraries imported


<H1> Establish the location of the folium map

In [126]:
madrid_address = 'Madrid, Madrid'
geolocator = Nominatim(user_agent="usera_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Madrid are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Madrid are 40.4167047, -3.7035825.


<H2> Initiate credentials for API

In [195]:
CLIENT_ID = 'YAQ4OGXJE0ILVST1RW3Q4EIYXQQQZQXBO10AVTNDXLO1PGLN'
CLIENT_SECRET = 'R50I00GDUZ1O3UUAHZVZKL405KBHD4ZHKZUEFD0RSUV0P0AQ'
VERSION = '20200122'
radius = 10000
LIMIT = 150
CATEGORY = '4bf58dd8d48988d163941735'  #category for parks in foursquare API
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YAQ4OGXJE0ILVST1RW3Q4EIYXQQQZQXBO10AVTNDXLO1PGLN
CLIENT_SECRET:R50I00GDUZ1O3UUAHZVZKL405KBHD4ZHKZUEFD0RSUV0P0AQ


<H3> Initiate general park info for Madrid

In [128]:
madrid_latitude = 40.4168 # Madrid latitude value
madrid_longitude = -3.7038 # Madrid longitude value

name = "Madrid" # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(name, 
                                                               madrid_latitude, 
                                                               madrid_longitude))

Latitude and longitude values of Madrid are 40.4168, -3.7038.


In [129]:
url = 'https://api.foursquare.com/v2/venues/explore?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CATEGORY,
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    madrid_latitude, 
    madrid_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&categoryId=4bf58dd8d48988d163941735&client_id=YAQ4OGXJE0ILVST1RW3Q4EIYXQQQZQXBO10AVTNDXLO1PGLN&client_secret=R50I00GDUZ1O3UUAHZVZKL405KBHD4ZHKZUEFD0RSUV0P0AQ&v=20200122&ll=40.4168,-3.7038&radius=10000&limit=150'

In [130]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2870cda2e538001b3aa8f1'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Madrid',
  'headerFullLocation': 'Madrid',
  'headerLocationGranularity': 'city',
  'query': 'park',
  'totalResults': 189,
  'suggestedBounds': {'ne': {'lat': 40.50680009000009,
    'lng': -3.5858092498128977},
   'sw': {'lat': 40.32679990999991, 'lng': -3.8217907501871027}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b8aa4c2f964a5201d7732e3',
       'name': 'Parque del Retiro',
       'location': {'address': 'Pl. de la Independencia, 7',
        'crossStreet': 'C. Alcalá',
        'lat': 40.41517879765204,
        'lng': -3.6814713478088374,
        'labeledLatLn

In [134]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [135]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name','venue.id', 'venue.categories','venue.location.postalCode', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,id,categories,postalCode,lat,lng
0,Parque del Retiro,4b8aa4c2f964a5201d7732e3,Park,28014,40.415179,-3.681471
1,Casa de Campo,4b8a698ef964a520ca6b32e3,Park,28011,40.422383,-3.740673
2,Parque de Madrid,560f7c49498e45c037137430,Park,NaN,40.416668,-3.688675
3,Jardines de Sabatini,4c1e4ede8b3aa5932bd99b5f,Garden,28013,40.419954,-3.713126
4,Campo del Moro,4adcda37f964a520103c21e3,Garden,28013,40.418781,-3.719302
...,...,...,...,...,...,...
95,Anfiteatro Romano (Auditorio del Parque Enriqu...,4f0d871ce4b06ab9f83e31be,Theater,28045,40.390252,-3.683800
96,Parque Forestal de Entrevias,4e000ee8e4cd609e49afced6,Park,28053,40.376040,-3.672781
97,Av del General Peron,51540221e4b012dfe6048ad4,Park,NaN,40.452016,-3.695054
98,Parque Caramuel,4de7711ad4c0086a95098d7e,Park,28011,40.407982,-3.728163


In [41]:
pd.unique(nearby_venues['categories'])

array(['Park', 'Garden', 'Monument / Landmark', 'Plaza', 'Palace',
       'Bridge', 'Opera House', 'Lake', 'Other Great Outdoors',
       'Neighborhood'], dtype=object)

In [42]:
pd.unique(nearby_venues['name'])

array(['Parque del Retiro', 'Parque de Madrid',
       'Parque Madrid Río (Bosque de Trepa)', 'Jardines de Sabatini',
       'Paseo del Prado', 'Campo del Moro', 'Jardines de las Vistillas',
       'Parque de la Montaña', 'Plaza de la Independencia',
       'Jardines de Virgen del Puerto', 'Parque del Oeste',
       'Cuesta de Moyano', 'Madrid Río (Sector Norte)',
       'Madrid Río (Sector Central)', 'Parque de Atenas',
       'Parque de la Arganzuela', 'Madrid Río (Sector Sur)',
       'Bosque del Recuerdo',
       'Puerta de España de los Jardines del Buen Retiro',
       'Palacio de Cristal del Retiro', 'Templo de Debod',
       'Pradera de San Isidro', 'Madrid Río (Legazpi)',
       'Puerta de Madrid de los Jardines del Buen Retiro',
       'Parque de Santander', 'Parque de La Cornisa',
       'Madrid Río - Salón de Pinos', 'Playa de Madrid',
       'Parque de la Bombilla', 'Plaza de Oriente',
       'Parque Villa de Paris', 'Paseo de la Ribera del Manzanares',
       'Palacio Rea

<h2> Intiate API info for Madrid neighborhoods

In [196]:
CATEGORY2 = '530e33ccbcbc57f1066bbfe4' # State and Municipality

#'4bf58dd8d48988d1fd931735' First is for metro stations.  Second is for "neighborhoods"  4f2a25ac4b909258e854f55f
#530e33ccbcbc57f1066bbfe4

In [201]:
url = 'https://api.foursquare.com/v2/venues/explore?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CATEGORY2,
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    madrid_latitude, 
    madrid_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&categoryId=530e33ccbcbc57f1066bbfe4&client_id=YAQ4OGXJE0ILVST1RW3Q4EIYXQQQZQXBO10AVTNDXLO1PGLN&client_secret=R50I00GDUZ1O3UUAHZVZKL405KBHD4ZHKZUEFD0RSUV0P0AQ&v=20200122&ll=40.4168,-3.7038&radius=10000&limit=150'

In [202]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e28799702a1720028acd5e9'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Madrid',
  'headerFullLocation': 'Madrid',
  'headerLocationGranularity': 'city',
  'query': 'states municipalities',
  'totalResults': 135,
  'suggestedBounds': {'ne': {'lat': 40.50680009000009,
    'lng': -3.5858092498128977},
   'sw': {'lat': 40.32679990999991, 'lng': -3.8217907501871027}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4de3d42fae60e7f3ac1031d3',
       'name': 'Barrio de Santa Eugenia',
       'location': {'lat': 40.38222605327716,
        'lng': -3.606691119192689,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.38222605327716,


<h2> Build table of Madrid neighborhoods with coordinates

In [203]:
venues = results['response']['groups'][0]['items']
    
barrios = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
barrios =barrios.loc[:, filtered_columns]

# filter the category for each row
barrios['venue.categories'] = barrios.apply(get_category_type, axis=1)

# clean columns
barrios.columns = [col.split(".")[-1] for col in barrios.columns]

barrios

,name,categories,lat,lng
0,Barrio de Santa Eugenia,Neighborhood,40.382226,-3.606691
1,Madrid,City,40.416932,-3.701550
2,Barrio de Sol,Neighborhood,40.416878,-3.703659
3,Madrid de los Austrias,Neighborhood,40.415391,-3.708830
4,Cortes,Neighborhood,40.415243,-3.698498
...,...,...,...,...
95,Aluche,Neighborhood,40.387341,-3.760842
96,Almenara,Neighborhood,40.469872,-3.690918
97,Distrito de Ciudad Lineal,Neighborhood,40.437738,-3.637775
98,Colina,Neighborhood,40.457294,-3.650178


In [204]:
pd.unique(barrios['categories'])

array(['Neighborhood', 'City'], dtype=object)

In [205]:
barrios.groupby(['categories']).count()

,name,lat,lng
categories,,,
City,4,4,4
Neighborhood,96,96,96


In [206]:
pd.unique(barrios['name'])

array(['Barrio de Santa Eugenia', 'Madrid', 'Barrio de Sol',
       'Madrid de los Austrias', 'Cortes', 'Calle de Fuencarral', 'Opera',
       'Barrio Malasaña, Madrid', 'Barrio de Palacio', 'Embajadores',
       'Barrio de Chueca', 'Calle Valverde', 'La Latina Neighbourhood',
       'Barrio de Universidad', 'Barrio de las Letras',
       'Barrio de La Latina', 'Barrio de Lavapiés', 'Barrio de Malasaña',
       'Justicia', 'Barrio Conde Duque', 'Príncipe Pío', 'Atocha',
       'Serrano', 'Trafalgar', 'Barrio de Puerta del Ángel',
       'Colonia Los Angeles', 'Arapiles', 'Barrio de Salamanca',
       'Barrio de Chamberí', 'Barrio de Arapiles', 'Moncloa',
       'Berruguete', 'Chamberí', 'Imperial', 'Almagro',
       'Distrito de Moncloa', 'Bulevar de Ibiza', 'Delicias', 'Comillas',
       'Ibiza', 'Retiro', 'Castellana', 'Niño Jesús',
       'Ciudad de la Imagen', 'Calle diego de leon', 'Ríos Rosas',
       'Barrio de Pacífico', 'Barrio de Legazpi', 'Barrio de Los Metales',
       'Cua

In [207]:
# Get names of indexes for which column category is a city
cities = barrios[barrios['categories'] == 'City' ].index

# Delete these row indexes from dataFrame
barrios.drop(cities , inplace=True)

In [208]:
barrios

,name,categories,lat,lng
0,Barrio de Santa Eugenia,Neighborhood,40.382226,-3.606691
2,Barrio de Sol,Neighborhood,40.416878,-3.703659
3,Madrid de los Austrias,Neighborhood,40.415391,-3.708830
4,Cortes,Neighborhood,40.415243,-3.698498
5,Calle de Fuencarral,Neighborhood,40.420766,-3.701386
...,...,...,...,...
95,Aluche,Neighborhood,40.387341,-3.760842
96,Almenara,Neighborhood,40.469872,-3.690918
97,Distrito de Ciudad Lineal,Neighborhood,40.437738,-3.637775
98,Colina,Neighborhood,40.457294,-3.650178


In [209]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)



# add markers to the map
markers_colors = []
for lat, lon, poi in zip(barrios['lat'], barrios['lng'], barrios['name']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h1>Day 2
    get the geocoordinates of the 21 districts of madrid 
    https://en.wikipedia.org/wiki/Districts_of_Madrid
    Use those coordinates to discover how many parks are within 500 meters
    Profit